In [1]:
from tables import *

In [2]:
# Apaga todas as tabelas do banco
Base.metadata.drop_all(bind = engine)
# cria todas novamente
Base.metadata.create_all(engine)

In [3]:
# Adicionando atores
global genre1, genre2, movie1, movie2
with Session() as session:
    actor1 = actor(name="Robert Downey Jr.")
    actor2 = actor(name="Chris Evans")
    session.add(actor1)
    session.add(actor2)

    # Adicionando gêneros
    genre1 = movieGeneres(name="Action")
    genre2 = movieGeneres(name="Adventure")
    session.add(genre1)
    session.add(genre2)

     # Commitando as mudanças
    session.commit()

    # Adicionando filmes
    movie1 = movie(title="Iron Man", studio="Marvel Studios", genere_id = genre1.id)
    movie2 = movie(title="Captain America", studio="Marvel Studios", genere_id = genre2.id)
    session.add(movie1)
    session.add(movie2)

    # Associando atores a filmes
    movie_actor1 = movieActors(actor_id=actor1.id, movie_id=movie1.id)  # Robert Downey Jr. em Iron Man
    movie_actor2 = movieActors(actor_id=actor2.id, movie_id=movie2.id)
    session.add(movie_actor1)
    session.add(movie_actor2)

    director1 = director(name="Jon Favreau")
    director2 = director(name="Joe Johnston")
    session.add(director1)
    session.add(director2)

    movie_director1 = movieDirectors(movie_id=movie1.id, directors_id=movie1.id)  # Jon Favreau dirigiu Iron Man
    movie_director2 = movieDirectors(movie_id=movie2.id, directors_id=movie2.id)  # Joe Johnston dirigiu Captain America
    session.add(movie_director1)
    session.add(movie_director2)

    # Adicionando clientes
    client1 = client(name="Alice")
    client2 = client(name="Bob")

    session.add(client1)
    session.add(client2)
    session.commit()

    session1 = MovieSessions(movie_id=movie1.id, session_date=datetime(2024, 8, 1, 18, 0))
    session2 = MovieSessions(movie_id=movie2.id, session_date=datetime(2024, 7, 2, 18, 0))

    session.add(session1)
    session.add(session2)
    session.commit()

    review1 = sessionReview(client_id=client1.id, session_id = session1.id, rating=5, review="Amazing movie!")
    review2 = sessionReview(client_id=client2.id, session_id = session2.id, rating=4, review="Great movie!")

    session.add(review1)
    session.add(review2)
    session.commit()

    # Associando sessões a clientes
    session_client1 = sessionClient(session_id=session1.id, movie_id=movie1.id, client_id = client1.id)
    session_client2 = sessionClient(session_id=session2.id, movie_id=movie2.id, client_id = client2.id)
    session.add(session_client1)
    session.add(session_client2)

    session.commit()

In [4]:
def addActor(nome):
  with Session() as session:
    actor1 = actor(name=nome)
    session.add(actor1)
    session.commit()

def getActors():
  sql_query = '''
  SELECT *
  FROM actors;
  '''
  df = pd.read_sql(sql_query, engine)
  return df.head()

def addMovie(actor_name, studio_name, genere_name, movie_name, director_name, client_name, movie_Date = datetime.now(), ratin = 0, review = ""):
    with Session() as session:
        actor1 = session.query(actor).filter_by(name=actor_name).first()
        if actor1 == None:
            actor1 = actor(name=actor_name)
            session.add(actor1)
            session.commit()

        genere1 = session.query(movieGeneres).filter_by(name=genere_name).first()
        if genere1 == None:
            genere1 = movieGeneres(name=genere_name)
            session.add(genere1)
            session.commit()

        movie1 = session.query(movie).filter_by(title=movie_name).first()
        if movie1 == None:
            movie1 = movie(title=movie_name, genere_id = genere1.id, studio = studio_name)
            session.add(movie1)
            session.commit()

        # Associando atores a filmes
        movie_actor1 = movieActors(actor_id=actor1.id, movie_id=movie1.id)
        session.add(movie_actor1)

        director1 = session.query(director).filter_by(name=director_name).first()
        if director1 == None:
            director1 = director(name=director_name)
            session.add(director1)

        movie_director1 = movieDirectors(movie_id=movie1.id, directors_id=movie1.id)  # Jon Favreau dirigiu Iron Man
        session.add(movie_director1)
        session.commit()

        session1 = MovieSessions(movie_id=movie1.id, session_date=movie_Date)
        session.add(session1)
        session.commit()

         # Adicionando clientes
        client1 = session.query(client).filter_by(name=client_name).first()
        if client1 == None:
            client1 = client(name=client_name)
            session.add(client1)
            session.commit()

        review1 = sessionReview(client_id=client1.id, session_id = session1.id, rating=ratin, review=review)

        session.add(review1)
        session.commit()

def getMovies():
  sql_query = '''
  SELECT *
  FROM movie;
  '''
  df = pd.read_sql(sql_query, engine)
  return df.head()

def addDirector(nome):
  with Session() as session:
    director1 = director(name=nome)
    session.add(director1)
    session.commit()

def getDirectors():
  sql_query = '''
  SELECT *
  FROM directors;
  '''
  df = pd.read_sql(sql_query, engine)
  return df.head()

def addClient(nome):
  with Session() as session:
    client1 = client(name=nome)
    session.add(client1)
    session.commit()

def getClients():
  sql_query = '''
  SELECT *
  FROM client;
  '''
  df = pd.read_sql(sql_query, engine)
  return df.head()



def getSessionsReview():
  sql_query = '''
  SELECT *
  FROM sessionReview;
  '''
  df = pd.read_sql(sql_query, engine)
  return df.head()

def getSession(nome, dia, mes, hora):
  data_hora = datetime(2024, mes, dia, hora, 0)

  # Conversão para string no formato desejado
  data_hora_formatada = data_hora.strftime('%Y-%m-%d %H:%M:%S.%f')
  sql_query = '''
              SELECT movie.title, SessionReview.rating, SessionReview.review
              FROM MovieSessions
              JOIN movie ON movie.id = MovieSessions.movie_id
              JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
              Where movie.title = "{title}" and MovieSessions.session_date = "{data_str}"
              ORDER BY MovieSessions.session_date DESC;
              '''
  df = pd.read_sql(text(sql_query.format(title = nome,data_str= data_hora_formatada)), engine)
  if df.empty:
    return "Nenhuma sessão encontrada"
  return df.iloc[0]

# def getSession(nome, dia, mes, hora):
#   movie1 = session.query(movie).filter_by(title=nome).first()
#   if movie1 == None:
#       print("filme não cadastrado")
#       return
#   session1 = session.query(MovieSessions).filter_by(session_date=movie_Date, movie_id=movie1.id).first()
#   if session1 == None:
#       print("filme não cadastrado")
#       return
#   return session1

"""
def getSessionId(nome, dia, mes, hora):
  data_hora = datetime(2024, mes, dia, hora, 0)

  # Conversão para string no formato desejado
  data_hora_formatada = data_hora.strftime('%Y-%m-%d %H:%M:%S.%f')
  sql_query = '''
  SELECT MovieSessions.id
  FROM MovieSessions
  JOIN movie ON movie.id = MovieSessions.movie_id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  Where movie.title = "{title}" and MovieSessions.session_date = "{data_str}"
  ORDER BY MovieSessions.session_date DESC;
  '''
  df = pd.read_sql(text(sql_query.format(title = nome, data_str = data_hora_formatada)), engine)
  return df
"""

def getSessionId(nome, dia, mes, hora):
    data_hora = datetime(2024, mes, dia, hora, 0)
    data_hora_formatada = data_hora.strftime('%Y-%m-%d %H:%M:%S.%f')
    sql_query = '''
    SELECT MovieSessions.id
    FROM MovieSessions
    JOIN movie ON movie.id = MovieSessions.movie_id
    WHERE movie.title = "{title}" AND MovieSessions.session_date = "{data_str}"
    ORDER BY MovieSessions.session_date DESC;
    '''
    df = pd.read_sql(text(sql_query.format(title=nome, data_str=data_hora_formatada)), engine)
    if df.empty:
        return None
    return df.iloc[0]['id']

############################################################

# def addSession(movie_name, dia, mes, hora):
#   movie_Date = datetime(2024, dia, mes, hora, 0)
#   with Session() as session:
#     movie1 = session.query(movie).filter_by(title=movie_name).first()
#     if movie1 == None:
#         print("filme não cadastrado")
#         return
#     session1 = MovieSessions(movie_id=movie1.id, session_date=movie_Date)
#     session.add(session1)
#     session.commit()

def addSessionClient(client_name , dia, mes, hora):
    movie_Date = datetime(2024, dia, mes, hora, 0)
    with Session() as session:
      session1 = session.query(MovieSessions).filter_by(session_date=movie_Date).first()
      if session1 == None:
        print("sessão nao cadastrada")
        return
      movie1 = session.query(movie).filter_by(id=session1.movie_id).first()
      if movie1 == None:
          print("filme não cadastrado")
          return
      client1 = session.query(client).filter_by(name=client_name).first()
      if client1 == None:
          client1 = client(name=client_name)
          session.add(client1)
          session.commit()

      session_client1 = sessionClient(session_id=session1.id, movie_id=movie1.id, client_id = client1.id)
      session.add(session_client1)
      session.commit()

"""
def addSessionReview(nome_cliente, nome_filme, dia, mes, hora, nota, avaliacao):

  df = getSessionId(nome_filme, dia, mes, hora)
  with Session() as session:
    cliente = session.query(client).filter_by(name=nome_cliente).first()
    review = sessionReview(client_id=cliente.id, session_id=df.id, rating=nota, review=avaliacao)
    session.add(review)
    session.commit()
    """

def addSessionReview(nome_cliente, nome_filme, dia, mes, hora, nota, avaliacao):
    session_id = getSessionId(nome_filme, dia, mes, hora)
    if session_id is None:
        print("Sessão não encontrada.")
        return
    with Session() as session:
        cliente = session.query(client).filter_by(name=nome_cliente).first()
        if cliente is None:
            print("Cliente não encontrado.")
            return
        review = sessionReview(client_id=cliente.id, session_id=session_id, rating=nota, review=avaliacao)
        session.add(review)
        session.commit()

#############################################

def getSessionReview(nome_filme):
  sql_query = '''
  SELECT movie.title, sessionReview.review, MovieSessions.id
  FROM MovieSessions
  JOIN movie ON movie.id = MovieSessions.movie_id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  Where movie.title = "{title}"
  ORDER BY MovieSessions.session_date DESC;
  '''
  df = pd.read_sql(text(sql_query.format(title = nome_filme)), engine)
  return df.head()

def addMovieSession(filme, dia, mes, hora):
  session1 = MovieSessions(movie_id=movie1.id, session_date=datetime(2024, mes, dia, hora, 0))
  session.add(session1)
  session.commit()

def getMovieRank():
  sql_query = '''
  SELECT Movie.title, SessionReview.rating
  FROM MovieSessions
  JOIN Movie ON MovieSessions.movie_id = Movie.id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY SessionReview.rating DESC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

def getMovieOrdemAlfabetica():
  sql_query = '''
  SELECT Movie.title, SessionReview.rating
  FROM MovieSessions
  JOIN Movie ON MovieSessions.movie_id = Movie.id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY Movie.title ASC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

def getMovieOrdemCronologica():
  sql_query = '''
  SELECT movie.title, sessionReview.review, MovieSessions.session_date
  FROM MovieSessions
  JOIN movie ON movie.id = MovieSessions.movie_id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY MovieSessions.session_date DESC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

Questão 1: O programa permite cadastrar um filme com todos os detalhes solicitados (nome, estúdio, nota, comentário, diretor(es), ator(es) etc.)?

In [5]:
def addMovie(actor_name, studio_name, genere_name, movie_name, director_name, client_name, movie_Date = datetime.now(), ratin = 0, review = ""):
    with Session() as session:
        actor1 = session.query(actor).filter_by(name=actor_name).first()
        if actor1 == None:
            actor1 = actor(name=actor_name)
            session.add(actor1)
            session.commit()

        genere1 = session.query(movieGeneres).filter_by(name=genere_name).first()
        if genere1 == None:
            genere1 = movieGeneres(name=genere_name)
            session.add(genere1)
            session.commit()

        movie1 = session.query(movie).filter_by(title=movie_name).first()
        if movie1 == None:
            movie1 = movie(title=movie_name, genere_id = genere1.id, studio = studio_name)
            session.add(movie1)
            session.commit()

        # Associando atores a filmes
        movie_actor1 = movieActors(actor_id=actor1.id, movie_id=movie1.id)
        session.add(movie_actor1)

        director1 = session.query(director).filter_by(name=director_name).first()
        if director1 == None:
            director1 = director(name=director_name)
            session.add(director1)

        movie_director1 = movieDirectors(movie_id=movie1.id, directors_id=movie1.id)  # Jon Favreau dirigiu Iron Man
        session.add(movie_director1)
        session.commit()

        session1 = MovieSessions(movie_id=movie1.id, session_date=movie_Date)
        session.add(session1)
        session.commit()

         # Adicionando clientes
        client1 = session.query(client).filter_by(name=client_name).first()
        if client1 == None:
            client1 = client(name=client_name)
            session.add(client1)
            session.commit()

        review1 = sessionReview(client_id=client1.id, session_id = session1.id, rating=ratin, review=review)

        session.add(review1)
        session.commit()

Questão 2: O programa permite cadastrar uma sessão, incluindo qual filme, as pessoas que assistiram junto e o comentário?

In [6]:
def addSession(movie_name,movie_Date):
  with Session() as session:
    movie1 = session.query(movie).filter_by(title=movie_name).first()
    if movie1 == None:
        print("filme não cadastrado")
        return
    session1 = MovieSessions(movie_id=movie1.id, session_date=movie_Date)
    session.add(session1)
    session.commit()

def addSessionClient(client_name ,datetime):
    with Session() as session:
      session1 = session.query(MovieSessions).filter_by(session_date=datetime).first()
      if session1 == None:
        print("sessão nao cadastrada")
        return
      movie1 = session.query(movie).filter_by(id=session1.movie_id).first()
      if movie1 == None:
          print("filme não cadastrado")
          return
      client1 = session.query(client).filter_by(name=client_name).first()
      if client1 == None:
          client1 = client(name=client_name)
          session.add(client1)
          session.commit()

      session_client1 = sessionClient(session_id=session1.id, movie_id=movie1.id, client_id = client1.id)
      session.add(session_client1)
      session.commit()

Questão 3: O programa lista corretamente todos os filmes por ordem decrescente de nota (mostrando nota e nome do filme)?

In [7]:
def getMovieRank():
  sql_query = '''
  SELECT Movie.title, SessionReview.rating
  FROM MovieSessions
  JOIN Movie ON MovieSessions.movie_id = Movie.id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY SessionReview.rating DESC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

Questão 4: O programa lista corretamente todos os filmes por ordem alfabética (mostrando nome e nota do filme)?

In [8]:
def getMovieOrdemAlfabetica():
  sql_query = '''
  SELECT Movie.title, SessionReview.rating
  FROM MovieSessions
  JOIN Movie ON MovieSessions.movie_id = Movie.id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY Movie.title ASC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

Questão 5: Funcionalidade de Listar Sessões por Ordem Cronológica Descendente OK

In [9]:
def getMovieOrdemCronologica():
  sql_query = '''
  SELECT movie.title, sessionReview.review, MovieSessions.session_date
  FROM MovieSessions
  JOIN movie ON movie.id = MovieSessions.movie_id
  JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
  ORDER BY MovieSessions.session_date DESC;
  '''
  df = pd.read_sql(sql_query, engine)
  return df

In [10]:
addMovie("McQueen", "Pixar", "Corrida", "Carros 3", "Lewis Hamilton", "Max Verstappen", ratin=5, review="Muito interessante")

In [11]:
getSessionReview("Carros 3")

,title,review,id
0,Carros 3,Muito interessante,3


In [12]:
getMovieRank()

,title,rating
0,Iron Man,5
1,Carros 3,5
2,Captain America,4


In [13]:
getMovieOrdemAlfabetica()

,title,rating
0,Captain America,4
1,Carros 3,5
2,Iron Man,5


In [14]:
getMovieOrdemCronologica()

,title,review,session_date
0,Iron Man,Amazing movie!,2024-08-01 18:00:00.000000
1,Captain America,Great movie!,2024-07-02 18:00:00.000000
2,Carros 3,Muito interessante,2024-06-23 22:26:48.277250


In [15]:
sql_query = '''
SELECT Movie.title, SessionReview.rating
FROM MovieSessions
JOIN Movie ON MovieSessions.movie_id = Movie.id
JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
ORDER BY SessionReview.rating DESC;
'''
df = pd.read_sql(sql_query, engine)

df.head()

,title,rating
0,Iron Man,5
1,Carros 3,5
2,Captain America,4


In [16]:
"""
sql_query = '''
SELECT Movie.title, SessionReview.rating
FROM MovieSessions
JOIN Movie ON MovieSessions.movie_id = Movie.id
JOIN SessionReview ON MovieSessions.sessionReview_id = SessionReview.id
ORDER BY Movie.title;
'''
df = pd.read_sql(sql_query, engine)

df.head()
"""

"\nsql_query = '''\nSELECT Movie.title, SessionReview.rating\nFROM MovieSessions\nJOIN Movie ON MovieSessions.movie_id = Movie.id\nJOIN SessionReview ON MovieSessions.sessionReview_id = SessionReview.id\nORDER BY Movie.title;\n'''\ndf = pd.read_sql(sql_query, engine)\n\ndf.head()\n"

In [17]:
sql_query = '''
SELECT movie.title, sessionReview.review, MovieSessions.session_date
FROM MovieSessions
JOIN movie ON movie.id = MovieSessions.movie_id
JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
ORDER BY MovieSessions.session_date DESC;
'''
df = pd.read_sql(sql_query, engine)

df.head()

,title,review,session_date
0,Iron Man,Amazing movie!,2024-08-01 18:00:00.000000
1,Captain America,Great movie!,2024-07-02 18:00:00.000000
2,Carros 3,Muito interessante,2024-06-23 22:26:48.277250


In [18]:
from sqlalchemy import create_engine, Column, Integer, String, MetaData, ForeignKey, DateTime, text

sql_query = '''
SELECT movie.title, sessionReview.review, MovieSessions.session_date
FROM MovieSessions
JOIN movie ON movie.id = MovieSessions.movie_id
JOIN SessionReview ON MovieSessions.id = SessionReview.session_id
Where movie.title = "{title}"
ORDER BY MovieSessions.session_date DESC;
'''
df = pd.read_sql(text(sql_query.format(title = 'Iron Man')), engine)

df.head()

,title,review,session_date
0,Iron Man,Amazing movie!,2024-08-01 18:00:00.000000


In [22]:
while True:
    with Session() as session:
        print("\n----->  Menu  <----\n")
        print("\t0 - Sair")
        print("\t1 - Cadastrar filme e sessão")
        print("\t2 - Listar filmes por ordem de nota")
        print("\t3 - Listar filmes por ordem alfabética")
        print("\t4 - Listar sessões por ordem cronológica ")
        print("\t5 - Consultar dados de uma sessão")

        opcao = input("\n\tOpção: ")

        if opcao == '1':
            print("\n>>> Cadastro de filme e sessão")
            movie_name = input("\n\tNome do filme: ")
            ator_name = input("\n\tNome do ator: ")
            studio_name = input("\n\tNome do estudio: ")
            genere_name = input("\n\tGenero: ")
            director_name = input("\n\tNome do diretor: ")
            client_name = input("\n\tNome do cliente: ")
            dia = input("\n\tDia da sessão: ")
            mes = input("\n\tMes da sessão: ")
            hora = input("\n\tHora da sessão: ")
            movie_Date = datetime(2024, int(dia), int(mes), int(hora), 0)
            ratin = input("\n\tNota: ")
            review = input("\n\tReview: ")
            addMovie(ator_name, studio_name, genere_name, movie_name, director_name, client_name, movie_Date, ratin, review)


        elif opcao == '2':

            print("\n>>> Filmes por ordem decrescente de nota ")

            print(getMovieRank())

        elif opcao == '3':

            print("\n>>> Filmes por ordem alfabética")

            print(getMovieOrdemAlfabetica())

        elif opcao == '4':

            print("\n>>> Sessões por ordem cronológica")

            print(getMovieOrdemCronologica())

        elif opcao == '5':

            print("\n>>> Dados de uma sessão")
            dia = input("\n\tDia da sessão: ")
            mes = input("\n\tMes da sessão: ")
            hora = input("\n\tHora da sessão: ")
            movie_name = input("\n\tNome do filme: ")
            print(getSession(movie_name, int(mes), int(dia), int(hora)))

        elif opcao == '0':

            print("\tSaindo do programa...")

            break

        else:
            print("\tOpção inválida.\n\tEscolha outra opção do menu.")



----->  Menu  <----

	0 - Sair
	1 - Cadastrar filme e sessão
	2 - Listar filmes por ordem de nota
	3 - Listar filmes por ordem alfabética
	4 - Listar sessões por ordem cronológica 
	5 - Consultar dados de uma sessão


KeyboardInterrupt: Interrupted by user

In [20]:
sql_query = '''
SELECT *
FROM SessionReview;
'''
df = pd.read_sql(sql_query, engine)
df

,id,client_id,session_id,rating,review
0,1,1,1,5,Amazing movie!
1,2,2,2,4,Great movie!
2,3,3,3,5,Muito interessante
